* Sentiment analysis remains one of the key problems that has seen extensive application of natural language processing. 
* This time around, given the tweets from customers about various tech firms who manufacture and sell mobiles, computers, laptops, etc,
* The task is to identify if the tweets have a negative sentiment towards such companies or products.

* Details of the files : 

* train.csv - For training the models, we provide a labelled dataset of 7920 tweets. The dataset is provided in the form of a csv file with each line storing a tweet id, its label and the tweet.

* test.csv - The test data file contains only tweet ids and the tweet text with each tweet in a new line.

* sample_submission.csv - The exact format for a valid submission

In [ ]:
import pandas as pd 
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from gensim.models import Word2Vec
from numpy import asarray
from numpy import zeros
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import RegexpTokenizer
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
# general imports
import math
from bs4 import BeautifulSoup
import tensorflow as tf
import numpy as np
import skimage
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
import missingno as msno
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Load train data
tweet= pd.read_csv('../input/analytics-vidhya-identify-the-sentiments/train.csv')
tweet.head(5)

In [ ]:
#Load test data
test= pd.read_csv('../input/analytics-vidhya-identify-the-sentiments/test.csv')
test.head(5)

In [ ]:
print('There are {} rows and {} columns in train'.format(tweet.shape[0],tweet.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

In [ ]:
#let's check the class distribution.There are only two classes 0 and 1.
x=tweet.label.value_counts()
sns.barplot(x.index,x)
plt.gca().set_ylabel('samples')

In [ ]:
#Number of characters in tweets
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=tweet[tweet['label']==1]['tweet'].str.len()
ax1.hist(tweet_len,color='red')
ax1.set_title('Negative tweets')
tweet_len=tweet[tweet['label']==0]['tweet'].str.len()
ax2.hist(tweet_len,color='green')
ax2.set_title('Positive tweets')
fig.suptitle('Characters in tweets')
plt.show()

In [ ]:
#Number of words in a tweet
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=tweet[tweet['label']==1]['tweet'].str.split().map(lambda x: len(x))
ax1.hist(tweet_len,color='red')
ax1.set_title('Negative tweets')
tweet_len=tweet[tweet['label']==0]['tweet'].str.split().map(lambda x: len(x))
ax2.hist(tweet_len,color='green')
ax2.set_title('Positive tweets')
fig.suptitle('Words in a tweet')
plt.show()

In [ ]:
#Average word length in a tweet
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
word=tweet[tweet['label']==1]['tweet'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax1,color='red')
ax1.set_title('Negative')
word=tweet[tweet['label']==0]['tweet'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax2,color='green')
ax2.set_title('Positive')
fig.suptitle('Average word length in each tweet')

In [ ]:
#Common stopwords in tweets - First we will analyze tweets with class 0.
def create_corpus(target):
    corpus=[]
    
    for x in tweet[tweet['label']==target]['tweet'].str.split():
        for i in x:
            corpus.append(i)
    return corpus


In [ ]:
corpus=create_corpus(0)

dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1
        
top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10] 

x,y=zip(*top)
plt.bar(x,y)


In [ ]:
corpus=create_corpus(1)

dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1

top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10] 

x,y=zip(*top)
plt.bar(x,y)

In [ ]:
#Analyzing punctuations - Lets check class 1
plt.figure(figsize=(10,5))
corpus=create_corpus(1)

dic=defaultdict(int)
import string
special = string.punctuation
for i in (corpus):
    if i in special:
        dic[i]+=1
        
x,y=zip(*dic.items())
plt.bar(x,y)


In [ ]:
#move on to class 0.
plt.figure(figsize=(10,5))
corpus=create_corpus(0)

dic=defaultdict(int)
import string
special = string.punctuation
for i in (corpus):
    if i in special:
        dic[i]+=1
        
x,y=zip(*dic.items())
plt.bar(x,y,color='green')


In [ ]:
# Identify common words
counter=Counter(corpus)
most=counter.most_common()
x=[]
y=[]
for word,count in most[:40]:
    if (word not in stop) :
        x.append(word)
        y.append(count)

sns.barplot(x=y,y=x)

In [ ]:
#Ngram analysis
#we will do a bigram (n=2) analysis over the tweets.
#Let's check the most common bigrams in tweets.
def get_top_tweet_bigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
plt.figure(figsize=(10,5))
top_tweet_bigrams=get_top_tweet_bigrams(tweet['tweet'])[:10]
x,y=map(list,zip(*top_tweet_bigrams))
sns.barplot(x=y,y=x)

In [ ]:
#Data cleaning starts
df=pd.concat([tweet,test])
df.shape

In [ ]:
#Step 1 : Removing urls

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


In [ ]:
df['tweet_NoURL']=df['tweet'].apply(lambda x : remove_URL(x))

In [ ]:
df[['tweet','tweet_NoURL']].head(10)

In [ ]:
#Step 2 : Removing HTML tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)


In [ ]:
df['tweet_NoHTML']=df['tweet_NoURL'].apply(lambda x : remove_html(x))
df[['tweet','tweet_NoURL','tweet_NoHTML']].sample(10)

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
df['tweet_NoEMOJI']=df['tweet_NoHTML'].apply(lambda x: remove_emoji(x))
df[['tweet','tweet_NoURL','tweet_NoHTML','tweet_NoEMOJI']].head(10)

In [ ]:
#Step 4 :Removing punctuations
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [ ]:
df['tweet_NoPUNKT']=df['tweet_NoEMOJI'].apply(lambda x : remove_punct(x))
df[['tweet','tweet_NoURL','tweet_NoHTML','tweet_NoEMOJI','tweet_NoPUNKT']].head(10)

In [ ]:
#Step 5: transform to lowercase
df['tweet_tolower']= df['tweet_NoPUNKT'].str.lower()
df[['tweet_NoPUNKT','tweet_tolower']].head(10)

In [ ]:
#Step 6: remove stopwords 'n punctuation
sw = stopwords.words('english')

def transform_text(s):
    
    # remove html
    html=re.compile(r'<.*?>')
    s = html.sub(r'',s)
    
    # remove numbers
    s = re.sub(r'\d+', '', s)
    
    # remove punctuation
    # remove stopwords
    tokens = nltk.word_tokenize(s)
    
    new_string = []
    for w in tokens:
        # remove words with len = 2 AND stopwords
        if len(w) > 2 and w not in sw:
            new_string.append(w)
 
    s = ' '.join(new_string)
    s = s.strip()

    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    
    return s.strip()

df['tweet_sw'] = df['tweet_tolower'].apply(transform_text)
df[['tweet_NoPUNKT','tweet_tolower', 'tweet_sw']].head(10)

In [ ]:
#Step 7: lemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatizer_text(s):
    tokens = nltk.word_tokenize(s)
    
    new_string = []
    for w in tokens:
        lem = lemmatizer.lemmatize(w, pos="v")
        # exclude if lenght of lemma is smaller than 2
        if len(lem) > 2:
            new_string.append(lem)
    
    s = ' '.join(new_string)
    return s.strip()

df['tweet_lm'] = df['tweet_sw'].apply(lemmatizer_text)
df[['tweet_sw', 'tweet_lm']].head(20)

In [ ]:
#Step 8: transform to lowercase, select only alphabets and remove stopwords
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['tweet_lm']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word.lower() not in stop))]
        corpus.append(words)
    return corpus

corpus=create_corpus(df)

Data cleaning Completed

In [ ]:
#see the word cloud with treated text
# -ve wordcloud
df_neg = df[df['label']==1]['tweet_lm']

wordcloud1 = WordCloud(stopwords=STOPWORDS,
                      background_color='white',
                      width=2500,
                      height=2000
                      ).generate(" ".join(df_neg))

plt.figure(1,figsize=(15, 15))
plt.imshow(wordcloud1)
plt.axis('off')
plt.show()

In [ ]:
# +ve wordcloud
df_pos = df[df['label']==0]['tweet_lm']

wordcloud1 = WordCloud(stopwords=STOPWORDS,
                      background_color='white',
                      width=2500,
                      height=2000
                      ).generate(" ".join(df_pos))

plt.figure(1,figsize=(10, 10))
plt.imshow(wordcloud1)
plt.axis('off')
plt.show()

In [ ]:
MAX_LEN = max(len(x) for x in corpus)
print("MAX_LEN = ", MAX_LEN)

* Model Building starts 

> ****Approach 1 : Using GloVe Embeddings with Keras + Convolutional Layer

In [ ]:
# Modelling starts with GloVe pre-trained 100
embedding_dict={}
with open('../input/glove6b100dtxt/glove.6B.100d.txt','r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [ ]:
#Lets check unique words
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        print("Inside i greater than")
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        #print("i =" , i)
        embedding_matrix[i]=emb_vec


In [ ]:
#Baseline Model
model=Sequential()

embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=True)

model.add(embedding)
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

In [ ]:
LEARNING_RATE = 1e-4
optimizer = tf.keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Segregating train and test from corpus
train=tweet_pad[:tweet.shape[0]]
test=tweet_pad[tweet.shape[0]:]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train,tweet['label'].values,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Run the Model for 10 Epochs

In [ ]:
history=model.fit(X_train,y_train,batch_size=4,epochs=10,validation_data=(X_test,y_test),verbose=1)

> ****We got 88% accuracy with GloVe Embeddings (Keras+CONV1D) 

Approach 2 : Multinomial Naive Bayes

In [ ]:
# Train-Test Split starts ######

'''
Total rows in train - 7920 

label=1 : Total 2026
we can use the first 1418(70%) for training and 608(30%) remaining for testing
i.e. 2026=1418+608

label=0 : Total 5894
we can use the first 4125(70%) for training and 1769(30%) remaining for testing
i.e. 5894=4125+1769
'''

# Train dataset
pos_train = df[df['label']==0][['tweet_lm', 'label']].head(4125)
neg_train = df[df['label']==1][['tweet_lm', 'label']].head(1418)


# Test dataset
#pos_test = df[df['label']==0][['tweet_lm', 'label']].tail(1769)
#neg_test = df[df['label']==1][['tweet_lm', 'label']].tail(608)

pos_test = df[df['label']==0][['tweet_lm', 'label']].iloc[4125:5894]
neg_test = df[df['label']==1][['tweet_lm', 'label']].iloc[1418:2026]


# put all toghether 
train_df = pd.concat([pos_train, neg_train]).sample(frac = 1).reset_index(drop=True)
test_df = pd.concat([pos_test, neg_test]).sample(frac = 1).reset_index(drop=True)

train_df.head()
test_df.head()

print('There are {} rows and {} columns in train'.format(train_df.shape[0],train_df.shape[1]))
print('There are {} rows and {} columns in test'.format(test_df.shape[0],test_df.shape[1]))


X_train = train_df['tweet_lm']
X_test  = test_df['tweet_lm']
y_train = train_df['label']
y_test  = test_df['label']


###Prepare actual Test data without label for submission
test1 = df['tweet_lm'].iloc[7920:]

We will use SelectKBest to select Top 40k features****

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
NGRAM_RANGE = (1, 2)
TOP_K = 40000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 1

In [ ]:
vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
vectorizer.fit(X_train)
x_train = vectorizer.transform(X_train)
x_val   = vectorizer.transform(X_test)


#Select top 'k' of the vectorized features. top_k = 6000
selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
print("x_train.shape[1] , TOP_K", x_train.shape[1], TOP_K)
selector.fit(x_train, y_train)
x_train_1 = selector.transform(x_train)
x_val_1   = selector.transform(x_val)

x_train_1 = x_train_1.astype('float32')
x_val_1 = x_val_1.astype('float32')

In [ ]:
# Generic function for model building
def fit_and_test(classifier, X_train, y_train, X_test, y_test, only_return_accuracy=False):
  classifier.fit(X_train, y_train)
  y_hat = classifier.predict(X_test)
  print('accuracy:', accuracy_score(y_test, y_hat))
  if not only_return_accuracy:
    print('f1_score:', f1_score(y_test, y_hat,average='micro'))


In [ ]:
#MultinomialNB
mnb = MultinomialNB() #88.85% for binary=False, ngram_range=(1, 2)
fit_and_test(mnb, x_train_1, y_train, x_val_1, y_test)


> **We got 88.80% accuracy with Multinomial Naive Bayes and using N-gram = (1,2) with top 40k features**

In [ ]:
#Making our submission
sample_submission = pd.read_csv('../input/analytics-vidhya-identify-the-sentiments/sample_submission.csv')
test_all = vectorizer.transform(df['tweet_lm'].iloc[7920:].values)
test_all = selector.transform(test_all)

y_predict = (mnb.predict(test_all.toarray()) > 0.5).astype("int32")
sample_submission["label"] = y_predict
sample_submission.to_csv("./sample_submission-mnb.csv", index=False)
sample_submission.head()

> ****The submission achieved 88.94% Accuracy (Rank : 248 on Christmas 2020) for Twitter Sentiment Analysis competion in AnalyticsVidhya :) 

> **If you have liked my Kernel - Please UpVote**